In [1]:
import pandas as pd
import random
from itertools import permutations

### Emojis 😵‍

In [2]:
"""
extra emojis!!! (watch out for odd extra hidden characters, will add silly columns to your dataframe)
👺⚡🎯🛡️⭐🐍🔴🔵
🗡️⚔️🪓🛡️💥🔥🟥🟦🆘
🧌💀💩👾👹😈👿👽
⏸️☣️🦂🤢☠️💫✨➕👘👕🎽
🤺🥷🏼🧙‍♂️😵💤🛌🔮🐉☆🧪🕸🕸️⚠️❌🪖🧱🕷
⏸😵‍💫⛊🔰⛨🛡️🩸
"""

# rolls
red = '🟥'
blue = '🟦'
hit = '💥'
shield = '⛨'
lightning = '⚡'
SS = '🐉'
star = '☆'
goblin = '😈'

# effects
armour = '🧱'
KO = '🛌'
stun = '⏸'
poison = '🤢'
critical = '❌'
lethal = '💀'
slow = '🕸'
bleed = '🩸'

### Classes and Functions

In [3]:
class Red:
    POSSIBLE_RESULTS = [
        [SS],
        [hit, hit],
        [lightning, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = red
    
    def roll(self):
        return [self.colour] + random.choice(self.POSSIBLE_RESULTS)

    
class Blue(Red):
    POSSIBLE_RESULTS = [
        [shield],
        [shield],
        [shield],
        [shield],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit, star],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = blue

    
class Dice:
    def __init__(self, n_red=0, n_blue=0):
        self.n_red = n_red
        self.n_blue = n_blue
        self._dice = [Red() for _ in range(n_red)] + [Blue() for _ in range(n_blue)]
        self.current_roll = None
    
    def roll(self):
        result = []
        for die in self._dice:
            die_result = die.roll()
            result.append(die_result)
        self.current_roll = result
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        """zero indexed - 1st item in n_to_reroll=0"""
        reroll = self._dice[n_to_reroll].roll()
        self.current_roll[n_to_reroll] = reroll
        return self.current_roll
    
    
class Item:
    def __init__(self, n_red=0, n_blue=0, modifiers=None, effects=None, thorgren=False):
        self.dice = Dice(n_red, n_blue)
        self.modifiers = modifiers
        self.effects = effects
        self.current_roll = None
        self.current_totals = None
        self.thorgren = thorgren
        self.roll()
        
    def roll(self, focus=False):
        if self.dice:
            result = self.dice.roll()
        else:
            result = []
        for k,v in self.modifiers.items():
            modifier_list = ['modifiers']
            for _ in range(v):
                modifier_list.append(k)
        result.append(modifier_list)
        if focus:
            result.append(['focus', hit])
        self.current_roll = result    
        self.calc_totals()
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        if self.current_roll[n_to_reroll][0] == 'modifiers':
            print('cannot reroll modifiers')
            return self.current_roll
        self.current_roll[n_to_reroll] = self.dice.reroll(n_to_reroll)[n_to_reroll]
        self.calc_totals()
        return self.current_roll
    
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()
            
    def calc_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    if result != red and result != blue and result != 'modifiers':
                        final_results[result] = final_results.get(result, 0) + 1
        self.current_totals = final_results

    def print_totals(self):
        if self.current_totals == None:
            self.calc_totals()
        for result in self.current_totals.items():
            print(result)
        print()


class Weapon(Item):                      
    def results(self):
        """
        check all effects and list all potential outcomes, incl no effects
        there is definitely a way to make this more efficient, likely needs recursive search algo, leave for now
        """        
        # add an extra effect if self.thorgren == True
        effects = self.effects + [(lightning, hit)] if self.thorgren else self.effects
        
        fx = all_permutations(effects)
        results = []
        for r in self.current_roll:
            results += [x for x in r[1:]]
        results = ''.join(sorted(results, reverse=True))
        columns = list(set(hit + ''.join(sorted([x[1] for x in effects]))))  # always show hits
        output = pd.DataFrame(columns=columns)

        for f in fx:
            current_result = results 
            if len(f) > 0:
                for cost, effect in f:
                    current_result = current_result.replace(cost, effect, 1)

            counts = {col: current_result.count(col) for col in output.columns}
            output = pd.concat([output, pd.DataFrame([counts])])

        output = output.drop_duplicates().sort_values(hit, ascending=False).reset_index(drop=True)
        return output
            
            
# calculate permutations of all lengths
def all_permutations(lst):
    for i in range(len(lst)+1):
        for p in permutations(lst, r=i):
            yield p

# simulation
def simulation(obj, n_trials=100):
    # TODO: determine methodology for summing, maxing, avging, that is appropriate for summarising results
    #  add columns for flags are necessary to complete statistical analysis
    obj.roll()
    results = obj.results()
    for _ in range(n_trials-1):
        obj.roll()
        results = pd.concat([results, obj.results()])
    return results

# plotting functions
def plot_simulation_results(dataframe):
    # TODO: this
    pass


### Items

In [4]:
spear_kwargs = {
    'n_red': 2,
    'n_blue': 2,
    'modifiers': {hit: 1},
    'effects': [(lightning, armour), (lightning+lightning, bleed)],
    'thorgren': True
}

In [5]:
spear = Weapon(**spear_kwargs)

spear.roll(focus=True)

print('\nDice\n')
spear.print_dice()

print('Totals\n')
spear.print_totals()

print('Results')
display(spear.results())


Dice

['🟥', '😈']
['🟥', '💥', '⚡']
['🟦', '💥', '⚡']
['🟦', '💥', '☆']
['modifiers', '💥']
['focus', '💥']

Totals

('😈', 1)
('💥', 5)
('⚡', 2)
('☆', 1)
('focus', 1)

Results


,💥,🧱,🩸
0,6,0,0
1,6,1,0
2,5,0,0
3,5,1,0
4,5,0,1


In [6]:
%%time
df = simulation(spear, 1000)

Wall time: 7.27 s
